<a href="https://colab.research.google.com/github/Pilo1961/QuestionAnswer_System/blob/master/code/question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Preguntas y Respuestas
## Métodos Analíticos

In [0]:
import numpy as np
import pandas as pd
import json

## Lectura de datos

In [3]:
#load data from google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
path= "/content/drive/My Drive/data/QA/train-v2.0.json"
train=pd.read_json(path,orient='columns')
#path= "/content/drive/My Drive/data/QA/dev-v2.0.json"
#test=pd.read_json(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Datos

Los datos los leemos en un archivo json anidado de varios nivles. La estructura del archivo es la siguiente

+ Titulo
+ Paragraph 
    + Context
    + Qas
        + Id
        + Question
        + Answers
            + Text
            + Answer start
        + Is impossible
        + Plausible answers
            + Text
            + Answer start
  

En las siguientes celdas observamos los diferentes niveles de anidacióón de los datos.

Los datos contienen 130319 preguntas divididas en dos grandes ramas. Preguntas que tienen respuesta y preguntas que no tienen respuestas.


In [4]:
# Lectura de datos
with open(path) as f: 
    d = json.load(f) 

test = pd.json_normalize(d['data'])  
test.shape
test.head()

,title,paragraphs
0,Beyoncé,[{'qas': [{'question': 'When did Beyonce start...
1,Frédéric_Chopin,"[{'qas': [{'question': ""What was Frédéric's na..."
2,Sino-Tibetan_relations_during_the_Ming_dynasty,[{'qas': [{'question': 'Who were Wang Jiawei a...
3,IPod,[{'qas': [{'question': 'Which company produces...
4,The_Legend_of_Zelda:_Twilight_Princess,[{'qas': [{'question': 'What category of game ...


El dataset de entrenamiento es un json anidado. En el primer nivel tiene una columna con el tíítulo y otra columna de json anidado llamada paragreaphs.

In [5]:
nycphil2=pd.json_normalize(data=d['data'],record_path=['paragraphs'],meta=['title'])
nycphil2.head()

,qas,context,title
0,[{'question': 'When did Beyonce start becoming...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé
1,"[{'question': 'After her second solo album, wh...",Following the disbandment of Destiny's Child i...,Beyoncé
2,"[{'question': 'In her music, what are some rec...","A self-described ""modern-day feminist"", Beyonc...",Beyoncé
3,[{'question': 'Beyonce's younger sibling also ...,"Beyoncé Giselle Knowles was born in Houston, T...",Beyoncé
4,[{'question': 'What town did Beyonce go to sch...,Beyoncé attended St. Mary's Elementary School ...,Beyoncé


Si abrimos paragraph tiene dos columnas:
  + qas - json anidado
  + context - Escrito con el contexto de la pregunta
  + title la obtenemos del nivel anterior

In [6]:
nycphil3=pd.json_normalize(data=d['data'],record_path=['paragraphs','qas'])#,meta=['title',['paragraph','context']])
nycphil3.head()

,question,id,answers,is_impossible,plausible_answers
0,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,"[{'text': 'in the late 1990s', 'answer_start':...",False,NaN
1,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,"[{'text': 'singing and dancing', 'answer_start...",False,NaN
2,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,"[{'text': '2003', 'answer_start': 526}]",False,NaN
3,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"[{'text': 'Houston, Texas', 'answer_start': 166}]",False,NaN
4,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,"[{'text': 'late 1990s', 'answer_start': 276}]",False,NaN


In [0]:
Al abrir la columna qas encontramos 5 columnas mas
 + Question
 + Id
 + Answers - que es un json anidado
 + is_impossible - con valores booleanos
 + plausible answers - Con NaN y los valores que tiene son json anidados


In [82]:
nycphil3=pd.json_normalize(data=d['data'],record_path=['paragraphs','qas','answers'])#,meta=['title',['paragraph','context']])
nycphil3.head()

,text,answer_start
0,in the late 1990s,269
1,singing and dancing,207
2,2003,526
3,"Houston, Texas",166
4,late 1990s,276


Cuando abrimos la columna answer encontramos dos columnas:
  + Text - texto que contiene la respuesta
  + Answer_start - Posicion de el contexto en el que comienza la respuesta

In [27]:
nycphil3=pd.json_normalize(data=d['data'],record_path=['paragraphs','qas'])#,'plausible_answers'])#,meta=['title',['paragraph','context']])
nycphil3.shape

(130319, 5)

# Preguntas con respuesta
Compilamos el dataframe llamado **df**.
* Contiene 7 columnas y todas las preguntas que cumplen las premisas.
* Contiene 86821 preguntas con respuesta dentreo del set de entrenamiento.
* No tiene valores nulos
* La **variable respuesta** es text, que contiene la respuesta a la pregunta y **answer start** que indica la posicion donde esta la respuesta dentro del contexto.
* La variable is_impossible no tiene variabilidad en preguntas con respuesta

In [54]:
df=pd.json_normalize(data=d['data'],
                        record_path=['paragraphs','qas','answers'],
                        record_prefix=None,
                        meta_prefix=None,
                        meta=['title',
                              ['paragraph','context'],
                              ['paragraph','qas','question'],
                              ['paragraph','qas','id'],
                              ['paragraph','qas','is_impossible']])

df.rename(columns={'paragraph.context':'context','paragraph.qas.question':'question','paragraph.qas.id':'id','paragraph.qas.is_impossible':'is_impossible'},inplace=True)
df.head()

,text,answer_start,title,context,question,id,is_impossible
0,in the late 1990s,269,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,False
1,singing and dancing,207,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,False
2,2003,526,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,False
3,"Houston, Texas",166,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,False
4,late 1990s,276,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,False


In [22]:
print("dimensiones df",df.shape)
print("Respuestas imposibles ", df["paragraph.qas.is_impossible"].sum())
print("Nas \n", df.isna().sum())


dimensiones df (86821, 7)
Respuestas imposibles  0
Nas 
 text                           0
answer_start                   0
title                          0
paragraph.context              0
paragraph.qas.question         0
paragraph.qas.id               0
paragraph.qas.is_impossible    0
dtype: int64


## Preguntas sin respuestas
La variable is_impossible boolanea representa las preguntas que no tienen respuesta que se deben de idetificar como parte de la versióón 2 de la base de datos.
Por el momento las vamos a ignorar, sin embargo se en las siguientes casillas se explora un poco la naturaleza de esos datos.
1. En los casos en los que plausible answers tiene valores distintos de Na la variable answer es vacía.
2. En los casos que la variable is_impossilbe tiene valores verdaderos la variable plausible_answers tiene valores distintos de NA.
3. Hay 43498 entradas en esta situacion.

In [38]:
df2=pd.json_normalize(data=d['data'],record_path=['paragraphs','qas'])
df2=df2[df2["is_impossible"]==True]
df2

,question,id,answers,is_impossible,plausible_answers
2075,What category of game is Legend of Zelda: Aust...,5a8d7bf7df8bba001a0f9ab1,[],True,"[{'text': 'action-adventure', 'answer_start': ..."
2076,What consoles can be used to play Australia Tw...,5a8d7bf7df8bba001a0f9ab2,[],True,"[{'text': 'GameCube and Wii', 'answer_start': ..."
2077,When was Australia Twilight launched in North ...,5a8d7bf7df8bba001a0f9ab3,[],True,"[{'text': 'November 2006', 'answer_start': 569}]"
2078,When could GameCube owners purchase Australian...,5a8d7bf7df8bba001a0f9ab4,[],True,"[{'text': 'November 2006', 'answer_start': 569}]"
2079,What year was the Legend of Zelda: Australian ...,5a8d7bf7df8bba001a0f9ab5,[],True,"[{'text': '2005', 'answer_start': 364}]"
...,...,...,...,...,...
130314,Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,[],True,"[{'text': 'matter', 'answer_start': 485}]"
130315,Who coined the term partonic matter?,5a7e070b70df9f001a87543a,[],True,"[{'text': 'Alfvén', 'answer_start': 327}]"
130316,What is another name for anti-matter?,5a7e070b70df9f001a87543b,[],True,"[{'text': 'Gk. common matter', 'answer_start':..."
130317,Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,[],True,"[{'text': 'a specifying modifier', 'answer_sta..."


In [45]:
# Todavio no lo saco pero no es necesario
pd.json_normalize(data=df2['plausible_answers'])

AttributeError: ignored

In [43]:
df2['plausible_answers'].iloc[1]

[{'answer_start': 194, 'text': 'GameCube and Wii'}]